In [38]:
# importing required modules
import PyPDF2
import re
import pandas as pd
import os
from bs4 import BeautifulSoup
import datetime

In [39]:
def get_new_PL(disposable):
    # EMI = P x R x (1+R)^N / [(1+R)^N-1] (where n = 60 months, r = 15% per annum and EMI = disposable)
    # disp = prq
    # q = a/b
    # a = (1 + (0.15/12))**60
    var1 = 1 + (0.15/12)
    var2 = 0.15/12
    var3 = var1**60
    
    value1 = (var3 - 1)*disposable
    value2 = var2*var3
    principal_amt = value1/value2
    return principal_amt

In [49]:
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

diff_month(datetime.date(2022,12,1),datetime.date(2021,10,1))

14

-----
# Recommendation

In [51]:
df = {'Products': {3: '1_CreditCard', 8: '1_CreditCard', 9: '1_CreditCard', 17: '1_CreditCard', 1: '3_PersonalLoan', 21: '3_PersonalLoan', 19: '3_PersonalLoan', 7: '3_PersonalLoan', 16: '3_PersonalLoan', 14: '3_PersonalLoan', 20: '4_Others', 18: '4_Others', 0: '4_Others', 12: '4_Others', 22: '4_Others', 5: '4_Others', 4: '4_Others', 2: '4_Others', 23: '4_Others'}, 'Loan Institution': {3: 'HDFC Bank Limited', 8: 'Axis Bank Ltd.', 9: 'Axis Bank Ltd.', 17: 'IDFC First Bank Ltd', 1: 'Vaibhav Vyapaar Private \nLimite', 21: 'OHMY Loan Private Limited', 19: 'IDFC First Bank Ltd', 7: 'Hero Fincorp Limited', 16: 'Fullerton India Credit \nCompany Ltd', 14: 'Tata Capital Financial Services \nLimite', 20: 'Kisetsu Saison Finance \n(India) Pvt Lt', 18: 'DMI Finance Private Limited', 0: 'Bajaj Finance Limited', 12: 'Manba Finance Ltd', 22: 'Kotak Mahindra Bank Ltd.', 5: 'Kotak Mahindra Bank Ltd.', 4: 'Kotak Mahindra Bank Ltd.', 2: 'Aditya Birla Finance', 23: 'InCred Financial Services \nLimite'}, 
    'date_opened': {
        3: datetime.date(2020, 1, 21), 8: datetime.date(2022, 2, 22), 9: datetime.date(2022, 6, 30), 17: datetime.date(2022, 5, 30), 1: datetime.date(2021, 8, 4), 21: datetime.date(2021, 3, 18), 19: datetime.date(2021, 11, 3), 7: datetime.date(2022, 9, 23), 16: datetime.date(2022, 7, 22), 14: datetime.date(2022, 1, 27), 20: datetime.date(2021, 9, 3), 18: datetime.date(2021, 10, 5), 0: datetime.date(2022, 9, 6), 12: datetime.date(2021, 3, 23), 22: datetime.date(2021, 4, 16), 5: datetime.date(2021, 12, 27), 4: datetime.date(2022, 1, 9), 2: datetime.date(2021, 10, 12), 23: datetime.date(2018, 8, 18)}, 'Sanction/Credit Limit': {3: 0, 8: 66000, 9: 16000, 17: 25000, 1: 6000, 21: 25000, 19: 4295, 7: 52000, 16: 60000, 14: 40000, 20: 30112021, 18: 10648, 0: 17528, 12: 84301, 22: 0, 5: 14198, 4: 12499, 2: 15000, 23: 1500}, 'Balance': {3: 955, 8: 34894, 9: 12180, 17: 23988, 1: 6000, 21: 3114, 19: 1457, 7: 52000, 16: 50607, 14: 26380, 20: 11499, 18: 3993, 0: 17528, 12: 53339, 22: 7418, 5: 3782, 4: 1467, 2: 2792, 23: 5}, 'EMI': {3: 47, 8: 1744, 9: 609, 17: 1199, 1: 0, 21: 3211, 19: 1484, 7: 5070, 16: 0, 14: 0, 20: 0, 18: 0, 0: 0, 12: 0, 22: 0, 5: 0, 4: 0, 2: 1448, 23: 1527}, 'Paid Principle': {3: 0, 8: 31106, 9: 3820, 17: 1012, 1: 0, 21: 21886, 19: 2838, 7: 0, 16: 9393, 14: 13620, 20: 30100522, 18: 6655, 0: 0, 12: 30962, 22: 0, 5: 10416, 4: 11032, 2: 12208, 23: 1495}, 'Delinquencies': {3: False, 8: False, 9: True, 17: False, 1: True, 21: False, 19: False, 7: False, 16: False, 14: False, 20: False, 18: False, 0: True, 12: False, 22: False, 5: False, 4: False, 2: False, 23: True}}
df = pd.DataFrame.from_dict(df)
# df.reset_index(inplace=True)
df

,Products,Loan Institution,date_opened,Sanction/Credit Limit,Balance,EMI,Paid Principle,Delinquencies
3,1_CreditCard,HDFC Bank Limited,2020-01-21,0,955,47,0,False
8,1_CreditCard,Axis Bank Ltd.,2022-02-22,66000,34894,1744,31106,False
9,1_CreditCard,Axis Bank Ltd.,2022-06-30,16000,12180,609,3820,True
17,1_CreditCard,IDFC First Bank Ltd,2022-05-30,25000,23988,1199,1012,False
1,3_PersonalLoan,Vaibhav Vyapaar Private \nLimite,2021-08-04,6000,6000,0,0,True
21,3_PersonalLoan,OHMY Loan Private Limited,2021-03-18,25000,3114,3211,21886,False
19,3_PersonalLoan,IDFC First Bank Ltd,2021-11-03,4295,1457,1484,2838,False
7,3_PersonalLoan,Hero Fincorp Limited,2022-09-23,52000,52000,5070,0,False
16,3_PersonalLoan,Fullerton India Credit \nCompany Ltd,2022-07-22,60000,50607,0,9393,False
14,3_PersonalLoan,Tata Capital Financial Services \nLimite,2022-01-27,40000,26380,0,13620,False


In [52]:
salary = 100000
if(salary <= 50000):
    FOIR = salary*0.5
elif(salary > 50000 and salary <= 150000):
    FOIR = salary*0.6
else:
    FOIR = salary*0.7
disposable = FOIR - df['EMI'].sum()

In [58]:
def get_top_up(df:pd.DataFrame):
    top_up_label = list()
    try:
        top_up = df.loc["3_PersonalLoan"]["Paid Principle"]
        top_up_label = ["Personal Loan",top_up]
    except KeyError:
        pass
    try:
        top_up = df.loc["5_AutoLoan"]["Paid Principle"]
        top_up_label = ["Auto Loan",top_up]
    except KeyError:
        pass
    try:
        top_up = df.loc["6_HomeLoan"]["Paid Principle"]
        top_up_label = ["Home Loan",top_up] 
    except KeyError:
        pass
    return top_up_label


In [63]:
recommendation = ""
recommendation_df = {"Recommendation":[],"New PL":[],"Top Up":[],"Reduce":[],"Remove":[]}
no_of_delinquecy = df['Delinquencies'].sum()
if(disposable>0):
    new_pl = int(get_new_PL(disposable))
    recommendation_df["New PL"].append(new_pl)
    recommendation+=f"You are eligible for a new Personal Loan of ₹{new_pl}"
else:
    recommendation+="You are not eligible for a new Personal Loan"

top_up_label = get_top_up(df)
if(len(top_up_label)>0 and top_up_label[1]>0):
    recommendation_df["Top Up"].append(top_up_label)
    recommendation+=f" and a Top Up of ₹{top_up_label[1]} on your {top_up_label[0]}"
else:
    recommendation+=". There is no Top Up available"
recommendation_df['Recommendation'].append(recommendation)
# recommendation_df = pd.DataFrame.from_dict(recommendation_df)

In [ ]:
# folder_loc = input("Enter folder location:")
folder_loc = "/Users/nilaygaitonde/Documents/E-RevBay/Automation/data"
# Create a list of all files in the folder
files = os.listdir(folder_loc)
# Create a list of all files with .pdf extension
pdf_files = [f for f in files if f.endswith('.pdf')]

In [ ]:
def get_index(firstIndex,string):
    return int(firstIndex)+int(len(string))

In [ ]:
def save_as_csv(data_df,pivot_df,csv,filename):
    if(csv):
        data_df.to_csv(f'{filename}_1.csv',index=False)
        pivot_df.to_csv(f'{filename}_2.csv',index=True)
    else:
        with pd.ExcelWriter(f'{filename}.xlsx') as writer:
            data_df.to_excel(writer,sheet_name='Sheet1',index=False)
            pivot_df.to_excel(writer,sheet_name='Sheet2',index=True)

In [ ]:
folder_loc = input("Enter folder location:")
files = os.listdir(folder_loc)
pdf_files = [f for f in files if f.endswith('.pdf')]

In [ ]:
# folder_loc = input("Enter folder location:")
# files = os.listdir(folder_loc)
# pdf_files = [f for f in files if f.endswith('.pdf')]
# complete_String = ""
# for file in pdf_files:
#     print("For name",file)
#     salary = int(input("enter salary:"))
#     pdfFileObj = open(folder_loc+"/"+file, 'rb')
# pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
# for i in range(pdfReader.numPages):
#     pageObj = pdfReader.getPage(i).extract_text()
#     complete_String += pageObj

In [ ]:
def create_loan(text,completeDF):
    completeDF = {"type":[],"institution":[],"date_opened":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[],"delinquecy":[]}
    countAcc = text.count("Acct # :")
    for _ in range(countAcc):
        delinquecy = False
        accountNoIndex = text.find("Acct # :")
        text = text[accountNoIndex+1:]
        openIndex = get_index(text.find('Open: '),'Open: ')
        openValue = text[openIndex:(text.find("Date Reported: "))].strip()

        dateOpenedIndex = get_index(text.find('Date Opened: '),'Date Opened: ')
        dateOpenedValue = text[dateOpenedIndex:(text.find("Type: "))].strip()
        try:
            date_object = datetime.strptime(dateOpenedValue, '%d-%m-%Y').date()
        except ValueError:
            pass
        # print(text)
        balanceIndex = get_index(text.find('Balance: '),'Balance: ')
        balance = text[balanceIndex:(text.find('Open:'))]
        if(balance[0] == "R"):
            balance = balance[4:]
        balance = int(balance.replace(',',''))
        # Find institution
        instiutionIndex = get_index(text.find('Institution : '),'Institution : ')
        instiutionName = text[instiutionIndex:(text.find('Past Due Amount'))-1]

        # Find type of loan
        typeIndex = get_index(text.find('Type: '),'Type: ')
        typeName = text[typeIndex:(text.find('Last Payment:'))-1]

        # TODO:::::::: FILL NAMES OF LOAN TYPES

        if(typeName not in ["Personal Loan","Business Loan","Credit Card","Housing Loan","Auto Loan",""]):
            typeName = "4_Others"
        elif(typeName == "Credit Card"):
            typeName = "1_CreditCard"
        elif(typeName == "Personal Loan"):
            typeName = "3_PersonalLoan"
        elif(typeName == "Business Loan"):
            typeName = "4_BusinessLoan"
        # Find EMI
        emiIndex = get_index(text.find('Monthly Payment Amount: '),'Monthly Payment Amount:')
        emiValue = text[emiIndex:(text.find('Credit Limit:'))-1]
        if(emiValue == ''):
            emiValue = 0
        else:
            emiValue = emiValue[4:]
            try:
                emiValue = int(emiValue.replace(',',''))
            except ValueError:
                emiValue = 0
        
        if(typeName == '1_CreditCard'):
            creditIndex = get_index(text.find('Credit Limit:'),"Credit Limit: Rs. ")
            creditLimit = text[creditIndex:text.find('Collateral Value')-1]
            try:
                sanction_credit = int(creditLimit.replace(',',''))
            except ValueError:
                sanction_credit = 0
            emiValue = balance*0.05
        else:
            sanctionIndex = get_index(text.find('Sanction Amount :'),"Sanction Amount : ")
            sanctionLimit = text[sanctionIndex:text.find('Reason:')]
            # print(sanctionLimit)
            if(sanctionLimit == ''):
                sanction_credit = 0
            else:
                sanctionLimit = sanctionLimit[4:]
                try:
                    sanction_credit = int(sanctionLimit.replace(',',''))
                except ValueError:
                    sanction_credit = 0
        AccountIndex = get_index(text.find('Account Status: '),'Account Status: ')
        AccountStatus = text[AccountIndex:text.find('Asset Classification')].strip()
        if(AccountStatus in [" ","Closed Account","Standard","Current Account"]):
            delinquecy = False
        else:
            delinquecy = True
        completeDF['balance'].append(int(balance))
        completeDF['institution'].append(instiutionName.strip())
        completeDF['type'].append(typeName.strip())
        completeDF['sanction_credit'].append(int(sanction_credit))
        completeDF['emi'].append(int(emiValue))
        completeDF['paid_principle'].append(int(sanction_credit-balance))
        completeDF['open'].append(openValue)
        completeDF['delinquecy'].append(delinquecy)
        completeDF['date_opened'].append(date_object)
    return completeDF

In [ ]:
# finalDF = {"type":[],"institution":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[],"delinquecy":[]}
# for file in pdf_files:
#     print("For name",file)
#     salary = input("enter salary:")
#     pdfFileObj = open(folder_loc+file, 'rb')
#     pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#     for i in range(pdfReader.numPages):
#         pageObj = pdfReader.getPage(i).extract_text()
#         on_page_2  = True
#         balance = 0
#         if(i == 0):
#             try:
#                 pageObj.index("Account Details :")
#                 on_page_2 = False
#             except ValueError:
#                 on_page_2 = True
#         if not on_page_2:
#             balanceIndex = get_index(pageObj.find('Balance: '),'Balance: ')
#             balance = pageObj[balanceIndex:(pageObj.find('Open:'))]
#             if(balance[0] == "R"):
#                 balance = balance[4:]
#             balance = int(balance.replace(',',''))
#         newDF = create_loan(pageObj,on_page_2,finalDF,balance)
#         [finalDF["type"].append(i) for i in newDF["type"]]
#         [finalDF["institution"].append(i) for i in newDF["institution"]]
#         [finalDF["sanction_credit"].append(i) for i in newDF["sanction_credit"]]
#         [finalDF["balance"].append(i) for i in newDF["balance"]]
#         [finalDF["emi"].append(i) for i in newDF["emi"]]
#         [finalDF["paid_principle"].append(i) for i in newDF["paid_principle"]]
#         [finalDF['open'].append(i) for i in newDF["open"]]
#         [finalDF['delinquecy'].append(i) for i in newDF["delinquecy"]]
        

In [ ]:
finalDF = {"type":[],"institution":[],"date_opened":[],"sanction_credit":[],"balance":[],"emi":[],"paid_principle":[],"open":[],"delinquecy":[]}
complete_String = ""
for file in pdf_files:
    salary = int(input(f"For name,{file}: \nEnter salary:"))
    pdfFileObj = open(folder_loc+"/"+file, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    for i in range(pdfReader.numPages):
        pageObj = pdfReader.getPage(i).extract_text()
        complete_String += pageObj
    riskIndex = get_index(complete_String.find("Equifax Risk Score 3.1 "), "Equifax Risk Score 3.1 ")
    riskValue = int(complete_String[riskIndex:(complete_String.find("1. "))].strip())
    print(riskValue)
    newDF = create_loan(complete_String,finalDF)
    [finalDF["type"].append(i) for i in newDF["type"]]
    [finalDF["institution"].append(i) for i in newDF["institution"]]
    [finalDF["sanction_credit"].append(i) for i in newDF["sanction_credit"]]
    [finalDF["balance"].append(i) for i in newDF["balance"]]
    [finalDF["emi"].append(i) for i in newDF["emi"]]
    [finalDF["paid_principle"].append(i) for i in newDF["paid_principle"]]
    [finalDF['open'].append(i) for i in newDF["open"]]
    [finalDF['delinquecy'].append(i) for i in newDF["delinquecy"]]
    [finalDF['date_opened'].append(i) for i in newDF["date_opened"]]
    data_df = pd.DataFrame.from_dict(finalDF)
    data_df['open'] = data_df['open'].map({
        'Yes': True,
        'No': False
    })
    data_df = data_df.loc[data_df['open']==True]
    # data_df.drop(['open'],axis=1,inplace=True)
    data_df.sort_values(by=['type'],ascending=True,inplace=True)
    # balance = 0 and no delinquecy is false
    # make filter for balance = 0 and delinquecy = false
    data_df = data_df.loc[(data_df['balance']!=0) | (data_df['delinquecy']==True)]
    data_df['paid_principle'] = data_df['paid_principle'].apply(lambda x: 0 if x<0 else x)
    data_df.groupby('type').sum()
    # salary = 50000
    df1=pd.DataFrame()
    if(salary <= 50000):
        FOIR = salary*0.5
    elif(salary > 50000 and salary <= 150000):
        FOIR = salary*0.6
    else:
        FOIR = salary*0.7
    delinquency = data_df['delinquecy'].sum()
    if(delinquency>0):
        delinquency = True
    else:
        delinquency = False
    disposable = FOIR - data_df['emi'].sum()
    print(disposable,FOIR,data_df['emi'].sum())
    sum_df = {
        'type': 'Total',
        'sanction_credit': data_df['sanction_credit'].sum(),
        'balance': data_df['balance'].sum(),
        'emi': data_df['emi'].sum(),
        'paid_principle': data_df['paid_principle'].sum(),
        'delinquecy': data_df['delinquecy'].sum(),
        'foir': FOIR,
        'disposable': disposable
    }
    sum_df = pd.DataFrame(sum_df,index=[0])
    pivot_df = data_df.groupby('type').sum()
    pivot_df = pivot_df.reset_index()
    pivot_df = pivot_df.append(sum_df,ignore_index=True)
    pivot_df.set_index('type',inplace=True)
    data_df.set_index('type',inplace=True)
    filename = os.path.basename(file).split('.')[0]
    save_as_csv(data_df = data_df,pivot_df = pivot_df,csv = True,filename=filename)

In [ ]:
# disp = prq
# q = a/b
# a = (1 + (0.15/12))**60
var1 = 1 + (0.15/12)
var2 = 0.15/12
var3 = var1**60

value1 = (var3 - 1)*disposable
print(disposable)
value2 = var2*var3
principal_amt = value1/value2
print(principal_amt)

In [ ]:
# -------------------OUTPUT-------------------
print("FOIR: ",FOIR)
print("emi",data_df['emi'].sum())
print("Disposable: ",disposable)
print("Delinuency:",delinquency)
# if(delinquency):
#     print("You have delinuency in your loan and you are not eligible for loan")
# else:
#     if(disposable>0):
#         print("You are eligible for loan")
#     else:
#         print("You are not eligible for loan")